## 数据读取



In [1]:
import pandas as pd

# 加载数据集
data_path = "lora_finetune.xlsx"
df = pd.read_excel(data_path)

# 查看数据前几行
print(df.head())

# 查看数据基本信息
print(df.info())

                            instruction  \
0  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   
1  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   
2  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   
3  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   
4  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   

                                               input  \
0  宠物出现以下症状：lethargy (嗜睡)、lameness (跛行)、neurologi...   
1  宠物出现以下症状：lameness (跛行)、neurological disorders ...   
2  宠物出现以下症状：lethargy (嗜睡)、fever (发烧)、tick fever (...   
3  宠物出现以下症状：lameness (跛行)、nasal discharge (鼻腔分泌物)...   
4  宠物出现以下症状：weight loss (体重减轻)、heart complication...   

                                              output  
0  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  
1  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  
2  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  
3  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  
4  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23999 entries, 0 to 2399

In [2]:
# 检查是否有缺失值
print(df.isnull().sum())

# 填充缺失值（如果有）
df.fillna({"instruction": "", "input": "", "output": ""}, inplace=True)

# 标准化文本格式
df["instruction"] = df["instruction"].str.strip()
df["input"] = df["input"].str.strip()
df["output"] = df["output"].str.strip()

# 再次检查数据
print(df.head())

instruction    0
input          0
output         0
dtype: int64
                            instruction  \
0  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   
1  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   
2  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   
3  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   
4  您好，我是一个专业的宠物健康顾问AI，可以通过分析症状来诊断宠物的疾病。   

                                               input  \
0  宠物出现以下症状：lethargy (嗜睡)、lameness (跛行)、neurologi...   
1  宠物出现以下症状：lameness (跛行)、neurological disorders ...   
2  宠物出现以下症状：lethargy (嗜睡)、fever (发烧)、tick fever (...   
3  宠物出现以下症状：lameness (跛行)、nasal discharge (鼻腔分泌物)...   
4  宠物出现以下症状：weight loss (体重减轻)、heart complication...   

                                              output  
0  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  
1  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  
2  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  
3  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  
4  根据提供的症状，宠物可能患有Tick fever (蜱热)，请及时就医，具体诊断和治疗方案请...  


In [3]:
from sklearn.model_selection import train_test_split

# 划分训练集和验证集
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# 保存为 JSON 文件（用于后续模型训练）
train_df.to_json("train_data.json", orient="records", lines=True)
val_df.to_json("val_data.json", orient="records", lines=True)


In [4]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 数据集tokenization

In [6]:
# 导入必要的库
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# 加载 JSON 数据集
train_dataset = load_dataset("json", data_files="train_data.json", split="train")
val_dataset = load_dataset("json", data_files="val_data.json", split="train")

# 加载预训练模型和分词器
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

# 配置 LoRA
lora_config = LoraConfig(
    r=8,  # LoRA 的秩（rank），控制低秩矩阵的大小
    lora_alpha=32,  # 缩放因子
    target_modules=["q_proj", "v_proj"],  # 根据模型架构选择目标模块
    lora_dropout=0.1,  # Dropout 比例
    bias="none",  # 是否调整偏置
    task_type="CAUSAL_LM"  # 任务类型
)

# 将模型包装为 LoRA 模型
model = get_peft_model(model, lora_config)

def tokenize_function(examples):
    # 确保 inputs 和 outputs 是字符串列表
    inputs = [str(instr) + " " + str(inp) for instr, inp in zip(examples["instruction"], examples["input"])]
    outputs = [str(out) for out in examples["output"]]
    
    # 对输入和输出进行编码
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=128, truncation=True, padding="max_length")
    
    # 将标签添加到模型输入中
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 对数据集进行 Tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# 使用 DataCollator 进行动态填充
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="max_length",
    max_length=512,  # 确保与 tokenize_function 中的 max_length 一致
    label_pad_token_id=-100  # 忽略填充部分的损失计算
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/21599 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [7]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir="./results_qwen-3b",  # 输出目录
    eval_strategy="epoch",           # 每个 epoch 进行一次评估
    learning_rate=1e-5,              # 学习率
    per_device_train_batch_size=1,  # 训练时每个设备的批量大小
    per_device_eval_batch_size=1,   # 评估时每个设备的批量大小
    num_train_epochs=10,             # 训练轮数
    weight_decay=0.01,              # 权重衰减
    save_strategy="steps",          # 按步数保存
    save_steps=500,                 # 每 500 步保存一次
    logging_dir="./logs",           # 日志目录
    logging_steps=50,               # 每 50 步记录一次日志
    fp16=True,                      # 使用混合精度训练（需要支持 GPU）
    gradient_accumulation_steps=4,  # 增加梯度累积步数
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train.select(range(1000)),  # 只使用前 1000 条数据
    eval_dataset=tokenized_val.select(range(500)),      # 只使用前 500 条数据
    tokenizer=tokenizer,
    data_collator=data_collator
)

import torch
torch.cuda.empty_cache()

# 开始训练
trainer.train()

/tmp/ipykernel_1545/1839693703.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,2.583500,2.317346
2,1.358600,1.318141
3,0.921500,0.906990
4,0.726200,0.717508
5,0.567500,0.556764
6,0.491400,0.483720
7,0.435900,0.439482
8,0.417800,0.418765
9,0.400500,0.408550
10,0.391500,0.401827


TrainOutput(global_step=2500, training_loss=1.485931826019287, metrics={'train_runtime': 2014.862, 'train_samples_per_second': 4.963, 'train_steps_per_second': 1.241, 'total_flos': 8.52976730112e+16, 'train_loss': 1.485931826019287, 'epoch': 10.0})

In [8]:

# 保存微调后的模型
model.save_pretrained("./results_qwen-3b")
tokenizer.save_pretrained("./results_qwen-3b")

('./results_qwen-3b/tokenizer_config.json',
 './results_qwen-3b/special_tokens_map.json',
 './results_qwen-3b/vocab.json',
 './results_qwen-3b/merges.txt',
 './results_qwen-3b/added_tokens.json',
 './results_qwen-3b/tokenizer.json')

In [10]:
import re
def generate_health_advice(instruction, input_text):
    # 将指令和输入拼接成完整输入
    full_input = f"{instruction} 输入：{input_text}"
    
    # 对输入进行编码
    inputs = tokenizer(
        full_input,
        return_tensors="pt",
        max_length=256,
        truncation=True,
        padding="max_length"
    )
    
    # 确保输入张量在正确的设备上
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # 使用 num_beams > 1 并移除 early_stopping
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=256,  # 增加最大生成长度
        num_beams=4,         # 使用束搜索
        do_sample=True,      # 启用采样
        temperature=0.7,     # 控制随机性
        top_p=0.9,           # 核采样
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # 解码生成的 token
    advice = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 提取生成的实际建议部分
    advice = advice.replace(full_input, "").strip()
    return advice

# 示例输入
example_instruction = (
    "您是经过培训的宠物个性化医疗保健代理，可以根据宠物的症状和体征，提供详细的诊断建议和护理指南。"
    "请分析以下输入中的症状，列出可能的原因，并给出具体的处理步骤和注意事项。"
)
example_input = (
    "最近的观察表明，动物类型:Dog 品种:Labrador 年龄:4岁 性别:Male 体重:25.0公斤, "
    "症状1:Fever 症状2:Lethargy 症状3:Appetite Loss 症状4:Vomiting 持续时间:3 days。"
    "食欲减退:Yes 呕吐:Yes 腹泻:No 咳嗽:No 呼吸困难:No 关节疼痛:No 皮肤病变:No 鼻分泌物:No 眼分泌物:No。"
    "我家的狗目前体温:39.5°C 心率:120，持续发热嗜睡"
)

# 分割输入文本为句子
sentences = [s.strip() for s in re.split(r'[。！？]', example_input) if s.strip()]

# 逐句生成健康建议
results = []
for sentence in sentences:
    advice = generate_health_advice(example_instruction, sentence)
    results.append((sentence, advice))

# 输出结果
for i, (sentence, advice) in enumerate(results):
    print(f"句子 {i+1}: {sentence}")
    print(f"健康建议: {advice}\n")

句子 1: 最近的观察表明，动物类型:Dog 品种:Labrador 年龄:4岁 性别:Male 体重:25.0公斤, 症状1:Fever 症状2:Lethargy 症状3:Appetite Loss 症状4:Vomiting 持续时间:3 days
健康建议: 输入：

根据您提供的信息，您提到的4岁成年雄性拉布拉多犬出现了持续3天的发热、嗜睡、食欲不振和呕吐的症状。这些症状可能由多种原因引起，包括感染性疾病（如病毒、细菌或寄生虫感染）、消化系统疾病（如胃肠道疾病或食物中毒）或代谢性疾病等。

可能的原因：
1. 感染性疾病
2. 消化系统疾病
3. 代谢性疾病
4. 其他疾病（例如肿瘤）

处理步骤:
1. 尽快带宠物去看兽医，进行详细检查和诊断。
2. 根据兽医的建议进行治疗，可能需要进行血液检查、尿液检查、影像学检查（如X光或超声波）等。
3. 遵医嘱给宠物喂食易消化的食物，如煮熟的鸡肉和米饭。
4. 保持宠物的环境清洁卫生，避免接触其他患病的宠物。
5. 观察宠物的症状是否有改善或恶化，及时与兽医沟通。

注意事项：
1、不要自行给宠物使用抗生素或其他药物，除非兽医明确指示。
2、密切观察宠物的体温、精神状态、排

句子 2: 食欲减退:Yes 呕吐:Yes 腹泻:No 咳嗽:No 呼吸困难:No 关节疼痛:No 皮肤病变:No 鼻分泌物:No 眼分泌物:No
健康建议: 根据您提供的症状，宠物可能患有消化系统疾病。可能的原因包括：

1. 消化不良
2. 肠炎
3. 胃炎
4. 食物过敏
5. 细菌感染
6. 病毒感染
7. 寄生虫感染
8. 药物过敏

处理步骤：
1. 尽量让宠物禁食12-24小时，以减轻胃肠道负担。
2. 给宠物提供清淡易消化的食物，如煮熟的鸡肉和白米饭。
3. 观察宠物的排便情况，如果出现血便或粘液便，应立即就医。
4. 如果宠物出现脱水症状，如尿量减少、口干舌燥、眼窝凹陷等，需要及时补充水分。
5. 如果症状持续或加重，应尽快带宠物就医。

注意事项：
- 在禁食期间，不要给宠物喂食任何食物或饮料，包括水。
- 在宠物恢复进食后，应逐渐恢复正常饮食，避免突然改变饮食习惯。
- 保持宠物的生活环境清洁卫生，定期清洁宠物的食盆和水盆。
- 注意观察宠物的其他症状，

句子 3: 我家的狗目前体温:39.5°C 心率:120，持续发热